<a href="https://colab.research.google.com/github/ClaudeJang/Duran-Duran-Returns/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC(1_21)_%EB%B0%8F_%EB%8F%84%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 처음 실행할 때 한 번만! 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools

!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null

!apt-get update -qq 2>&1 > /dev/null

!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth

auth.authenticate_user()

from oauth2client.client import GoogleCredentials

creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate


KeyboardInterrupt: ignored

In [0]:
!mkdir -p drive

!google-drive-ocamlfuse drive

In [9]:
cd drive

/root/drive


In [3]:
import pandas as pd
import numpy as np
import warnings 
import os # 운영체제에서 제공되는 기능을 파이썬에서 수행 
import keras
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [41]:
train = pd.read_csv('./Colab Notebooks/data/real_perfect_train.csv',encoding = 'utf-8',index_col= 0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [42]:
train.head()

,key,city,apartment_id,address_by_law,transaction_date,transaction_year_month,year_of_completion,apartment_building_count_in_sites,bathroom_count,exclusive_use_area,...,mortgage_loan,transaction_real_price,central,district,individual,cogeneration,gas,corridor,mixed,stairway
0,0,1,5584,1111017100,20,200601,1999,1,1.0,47.43,...,5.64,215000000,0,0,1,0,1,1,0,0
1,1,1,5584,1111017100,20,200601,1999,1,1.0,44.37,...,5.64,200000000,0,0,1,0,1,1,0,0
2,2,1,5059,1111017400,20,200601,1992,5,1.0,54.70,...,5.64,168000000,0,0,1,0,1,1,0,0
3,3,1,2816,1111017400,20,200601,1993,7,1.0,64.66,...,5.64,165000000,0,0,1,0,1,1,0,0
4,4,1,2816,1111017400,20,200601,1993,7,2.0,106.62,...,5.64,280000000,0,0,1,0,1,0,0,1


In [0]:
train = train.drop(columns = ['address_by_law','transaction_date','transaction_year_month','latitude','longitude','room_id','supply_area','total_household_count_of_area_type','household_loan','mean_loan','lowest_building_in_sites'])

In [0]:
train.head()

,key,city,apartment_id,year_of_completion,apartment_building_count_in_sites,bathroom_count,exclusive_use_area,floor,room_count,tallest_building_in_sites,...,mortgage_loan,transaction_real_price,central,district,individual,cogeneration,gas,corridor,mixed,stairway
0,0,1,5584,1999,1,1.0,47.43,6,1.0,8.0,...,5.64,215000000,0,0,1,0,1,1,0,0
1,1,1,5584,1999,1,1.0,44.37,8,2.0,8.0,...,5.64,200000000,0,0,1,0,1,1,0,0
2,2,1,5059,1992,5,1.0,54.70,8,2.0,14.0,...,5.64,168000000,0,0,1,0,1,1,0,0
3,3,1,2816,1993,7,1.0,64.66,11,2.0,15.0,...,5.64,165000000,0,0,1,0,1,1,0,0
4,4,1,2816,1993,7,2.0,106.62,7,4.0,15.0,...,5.64,280000000,0,0,1,0,1,0,0,1


In [0]:
train2 = train[['year_of_completion',
       'apartment_building_count_in_sites','bathroom_count',
       'exclusive_use_area', 'floor', 'room_count',
       'tallest_building_in_sites', 'total_household_count_in_sites',
       'total_parking_capacity_in_site','mortgage_loan',
       ]]
# 범주형 변수, key, city, apt_id를 제외한 연속형 변수만 

In [45]:
train2['parking_per_household'] = train2['total_parking_capacity_in_site'] / train2['total_household_count_in_sites'] 
# train2['parking_per_household']="%0.2f" % train2.parking_per_household
# train2.info()
train2['parking_per_household']=round(train2.parking_per_household,2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [46]:
train2.info()
train2 = train2.drop(columns = ['total_parking_capacity_in_site'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1601458 entries, 0 to 1605374
Data columns (total 11 columns):
year_of_completion                   1601458 non-null int64
apartment_building_count_in_sites    1601458 non-null int64
bathroom_count                       1601458 non-null float64
exclusive_use_area                   1601458 non-null float64
floor                                1601458 non-null int64
room_count                           1601458 non-null float64
tallest_building_in_sites            1601458 non-null float64
total_household_count_in_sites       1601458 non-null int64
total_parking_capacity_in_site       1601458 non-null float64
mortgage_loan                        1601458 non-null float64
parking_per_household                1601458 non-null float64
dtypes: float64(7), int64(4)
memory usage: 146.6 MB


In [29]:
train2.head()

,year_of_completion,apartment_building_count_in_sites,bathroom_count,exclusive_use_area,floor,room_count,tallest_building_in_sites,total_household_count_in_sites,mortgage_loan,parking_per_household
0,1999,1,1.0,47.43,6,1.0,8.0,136,5.64,1.20
1,1999,1,1.0,44.37,8,2.0,8.0,136,5.64,1.20
2,1992,5,1.0,54.70,8,2.0,14.0,585,5.64,1.54
3,1993,7,1.0,64.66,11,2.0,15.0,919,5.64,0.98
4,1993,7,2.0,106.62,7,4.0,15.0,919,5.64,0.98


In [0]:
# standardization

mean = train2.mean(axis=0)
train2 -= mean
std = train2.std(axis=0)
train2 /= std 

# test_data -= mean
# test_data /= std

In [48]:
train2.info()
train4 = train[['apartment_id','transaction_real_price','city','central','district','individual','cogeneration','gas','corridor','mixed','stairway']]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1601458 entries, 0 to 1605374
Data columns (total 10 columns):
year_of_completion                   1601458 non-null float64
apartment_building_count_in_sites    1601458 non-null float64
bathroom_count                       1601458 non-null float64
exclusive_use_area                   1601458 non-null float64
floor                                1601458 non-null float64
room_count                           1601458 non-null float64
tallest_building_in_sites            1601458 non-null float64
total_household_count_in_sites       1601458 non-null float64
mortgage_loan                        1601458 non-null float64
parking_per_household                1601458 non-null float64
dtypes: float64(10)
memory usage: 134.4 MB


In [0]:
train3 = pd.concat([train2,train4], axis = 1)

In [50]:
train3.head()

,year_of_completion,apartment_building_count_in_sites,bathroom_count,exclusive_use_area,floor,room_count,tallest_building_in_sites,total_household_count_in_sites,mortgage_loan,parking_per_household,...,transaction_real_price,city,central,district,individual,cogeneration,gas,corridor,mixed,stairway
0,0.042060,-0.839598,-1.157310,-1.144215,-0.574122,-2.932447,-1.459764,-0.926899,1.009664,0.220346,...,215000000,1,0,0,1,0,1,1,0,0
1,0.042060,-0.839598,-1.157310,-1.253498,-0.296118,-1.429483,-1.459764,-0.926899,1.009664,0.220346,...,200000000,1,0,0,1,0,1,1,0,0
2,-0.786868,-0.553712,-1.157310,-0.884577,-0.296118,-1.429483,-0.774081,-0.528882,1.009664,0.978990,...,168000000,1,0,0,1,0,1,1,0,0
3,-0.668450,-0.410768,-1.157310,-0.528869,0.120889,-1.429483,-0.659800,-0.232807,1.009664,-0.270541,...,165000000,1,0,0,1,0,1,1,0,0
4,-0.668450,-0.410768,0.814297,0.969673,-0.435120,1.576445,-0.659800,-0.232807,1.009664,-0.270541,...,280000000,1,0,0,1,0,1,0,0,1


In [51]:
train3.columns

Index(['year_of_completion', 'apartment_building_count_in_sites',
       'bathroom_count', 'exclusive_use_area', 'floor', 'room_count',
       'tallest_building_in_sites', 'total_household_count_in_sites',
       'mortgage_loan', 'parking_per_household', 'apartment_id',
       'transaction_real_price', 'city', 'central', 'district', 'individual',
       'cogeneration', 'gas', 'corridor', 'mixed', 'stairway'],
      dtype='object')

In [0]:
city_onehot = pd.get_dummies(train3.city) 


In [0]:
train3 = pd.concat(
	[train3, city_onehot],
	axis=1) 

In [55]:
train3.info()
# train3 = train3.rename(columns = {0:'busan',1:'seoul'})

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1601458 entries, 0 to 1605374
Data columns (total 23 columns):
year_of_completion                   1601458 non-null float64
apartment_building_count_in_sites    1601458 non-null float64
bathroom_count                       1601458 non-null float64
exclusive_use_area                   1601458 non-null float64
floor                                1601458 non-null float64
room_count                           1601458 non-null float64
tallest_building_in_sites            1601458 non-null float64
total_household_count_in_sites       1601458 non-null float64
mortgage_loan                        1601458 non-null float64
parking_per_household                1601458 non-null float64
apartment_id                         1601458 non-null int64
transaction_real_price               1601458 non-null int64
city                                 1601458 non-null int64
central                              1601458 non-null int64
district                   

In [0]:
train3 = train3[['apartment_id','year_of_completion','apartment_building_count_in_sites',
       'bathroom_count', 'exclusive_use_area', 'floor', 'room_count',
       'tallest_building_in_sites', 'total_household_count_in_sites',
       'mortgage_loan', 'parking_per_household',
       'central', 'district',
       'individual', 'cogeneration', 'gas', 'corridor', 'mixed', 'stairway','busan','seoul','transaction_real_price']]

In [58]:
train3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1601458 entries, 0 to 1605374
Data columns (total 22 columns):
apartment_id                         1601458 non-null int64
year_of_completion                   1601458 non-null float64
apartment_building_count_in_sites    1601458 non-null float64
bathroom_count                       1601458 non-null float64
exclusive_use_area                   1601458 non-null float64
floor                                1601458 non-null float64
room_count                           1601458 non-null float64
tallest_building_in_sites            1601458 non-null float64
total_household_count_in_sites       1601458 non-null float64
mortgage_loan                        1601458 non-null float64
parking_per_household                1601458 non-null float64
central                              1601458 non-null int64
district                             1601458 non-null int64
individual                           1601458 non-null int64
cogeneration               

In [0]:
train3.head() # 정규화를 끝낸 데이터셋 
train3.to_csv('./Colab Notebooks/data/Final_train.csv',encoding='utf-8') # 최종 데이터 전처리 트레인 

In [19]:
train3 = pd.read_csv('./Colab Notebooks/data/Final_train.csv',encoding = 'utf-8',index_col = 0)


/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [20]:
train3.head()

,apartment_id,year_of_completion,apartment_building_count_in_sites,bathroom_count,exclusive_use_area,floor,room_count,tallest_building_in_sites,total_household_count_in_sites,mortgage_loan,parking_per_household,central,district,individual,cogeneration,gas,corridor,mixed,stairway,transaction_real_price
0,5584,0.042060,-0.839598,-1.157310,-1.144215,-0.574122,-2.932447,-1.459764,-0.926899,1.009664,0.220346,0,0,1,0,1,1,0,0,215000000
1,5584,0.042060,-0.839598,-1.157310,-1.253498,-0.296118,-1.429483,-1.459764,-0.926899,1.009664,0.220346,0,0,1,0,1,1,0,0,200000000
2,5059,-0.786868,-0.553712,-1.157310,-0.884577,-0.296118,-1.429483,-0.774081,-0.528882,1.009664,0.978990,0,0,1,0,1,1,0,0,168000000
3,2816,-0.668450,-0.410768,-1.157310,-0.528869,0.120889,-1.429483,-0.659800,-0.232807,1.009664,-0.270541,0,0,1,0,1,1,0,0,165000000
4,2816,-0.668450,-0.410768,0.814297,0.969673,-0.435120,1.576445,-0.659800,-0.232807,1.009664,-0.270541,0,0,1,0,1,0,0,1,280000000


In [0]:
test = pd.read_csv('./Colab Notebooks/data/real_perfect_test.csv', encoding='utf-8',index_col = 0)


In [21]:
test.head()

,key,city,apartment_id,address_by_law,transaction_date,transaction_year_month,year_of_completion,apartment_building_count_in_sites,bathroom_count,exclusive_use_area,...,mortgage_loan,transaction_real_price,central,district,individual,cogeneration,gas,corridor,mixed,stairway
1568913,462533,1,3751,1168010400,31,200912,1984,4,1.0,83.58,...,5.90,0,0,1,0,1,0,1,0,0
1568914,764018,1,14029,1111013700,10,201304,1968,1,0.0,107.17,...,3.86,0,0,0,1,0,1,1,0,0
1568915,813528,1,12304,1111017400,30,201309,1968,4,1.0,36.17,...,3.82,0,0,0,1,0,1,0,0,1
1568916,845097,1,22241,1150010200,10,201312,2007,1,2.0,45.67,...,3.74,0,0,0,1,0,1,0,0,1
1568917,856338,1,316,1135010600,10,201401,1988,2,1.0,41.85,...,3.75,0,0,0,1,0,1,0,0,1


In [0]:
test = test.drop(columns = ['address_by_law','transaction_date','transaction_year_month','latitude','longitude','room_id','supply_area','total_household_count_of_area_type','household_loan','mean_loan','lowest_building_in_sites'])

In [24]:
test2 = test[['year_of_completion',
       'apartment_building_count_in_sites','bathroom_count',
       'exclusive_use_area', 'floor', 'room_count',
       'tallest_building_in_sites', 'total_household_count_in_sites',
       'total_parking_capacity_in_site','mortgage_loan',
       ]]
test2['parking_per_household'] = test2['total_parking_capacity_in_site'] / test2['total_household_count_in_sites'] 
test2['parking_per_household']= round(test2.parking_per_household,2)
test2 = test2.drop(columns = ['total_parking_capacity_in_site'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
mean = train2.mean(axis=0)
# train2 -= mean
std = train2.std(axis=0)
# train2 /= std 

test2 -= mean
test2 /= std

In [31]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3918 entries, 1568913 to 1605375
Data columns (total 10 columns):
year_of_completion                   3918 non-null float64
apartment_building_count_in_sites    3918 non-null float64
bathroom_count                       3918 non-null float64
exclusive_use_area                   3918 non-null float64
floor                                3918 non-null float64
room_count                           3918 non-null float64
tallest_building_in_sites            3918 non-null float64
total_household_count_in_sites       3918 non-null float64
mortgage_loan                        3918 non-null float64
parking_per_household                3918 non-null float64
dtypes: float64(10)
memory usage: 336.7 KB


In [0]:
test4 = test[['apartment_id','transaction_real_price','city','central','district','individual','cogeneration','gas','corridor','mixed','stairway']]

In [0]:
test3 = pd.concat([test2,test4], axis = 1)
city_onehot = pd.get_dummies(test3.city) 
test3 = pd.concat(
	[test3, city_onehot],
	axis=1) 

In [37]:
# test3.info()
# test3 = test3.rename(columns = {0:'busan',1:'seoul'})
test3.columns

Index(['year_of_completion', 'apartment_building_count_in_sites',
       'bathroom_count', 'exclusive_use_area', 'floor', 'room_count',
       'tallest_building_in_sites', 'total_household_count_in_sites',
       'mortgage_loan', 'parking_per_household', 'apartment_id',
       'transaction_real_price', 'city', 'central', 'district', 'individual',
       'cogeneration', 'gas', 'corridor', 'mixed', 'stairway', 'busan',
       'seoul'],
      dtype='object')

In [0]:
test3 = test3[['apartment_id','year_of_completion', 'apartment_building_count_in_sites',
       'bathroom_count', 'exclusive_use_area', 'floor', 'room_count',
       'tallest_building_in_sites', 'total_household_count_in_sites',
       'mortgage_loan', 'parking_per_household', 
       'city', 'central', 'district', 'individual',
       'cogeneration', 'gas', 'corridor', 'mixed', 'stairway', 'busan',
       'seoul','transaction_real_price']]

In [56]:
test3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3918 entries, 1568913 to 1605375
Data columns (total 23 columns):
apartment_id                         3918 non-null int64
year_of_completion                   3918 non-null float64
apartment_building_count_in_sites    3918 non-null float64
bathroom_count                       3918 non-null float64
exclusive_use_area                   3918 non-null float64
floor                                3918 non-null float64
room_count                           3918 non-null float64
tallest_building_in_sites            3918 non-null float64
total_household_count_in_sites       3918 non-null float64
mortgage_loan                        3918 non-null float64
parking_per_household                3918 non-null float64
city                                 3918 non-null int64
central                              3918 non-null int64
district                             3918 non-null int64
individual                           3918 non-null int64
cogenerat

In [0]:
test3.to_csv('./Colab Notebooks/data/Final_test.csv',encoding='utf-8') # 최종 데이터 전처리 트레인 